In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from os import chdir
import numpy as np
import pickle
import scipy.stats as stats


In [32]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    #clip outliers to 1st and 99th percentile
    features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=['liveness', 'instrumentalness', 'acousticness', 'speechiness','loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [33]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [34]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [35]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [36]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [37]:
emotionsDF = makeCategorical(emotionsDF)

In [38]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [39]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [40]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [41]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [42]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [43]:
rawfeaturesDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1.482419,-0.322342,-0.065799,-0.665392,-1.283231,2.700000,-0.958541,-0.489430,2.700000,-0.552893,0.940143,0.272427,0.173859
1,0.545890,0.371919,1.318501,0.064003,-1.283231,0.657598,-0.245604,-0.492971,1.000754,0.192677,0.071470,-0.199505,0.173859
2,0.851129,-0.735754,-1.173239,1.136073,0.779283,-0.530916,-0.967633,2.350764,-0.477927,-1.311172,0.247995,-1.125231,0.173859
3,-0.820748,-0.342998,-1.173239,-1.224490,0.779283,1.695751,-0.824045,-0.492970,-0.469337,-0.163163,-0.935335,-1.642980,0.173859
4,-0.113148,0.861786,0.211061,0.637965,-1.283231,-0.224387,-0.957888,-0.492977,1.425940,-0.158927,1.810239,-1.999696,0.173859


In [44]:
y = balancedDF['mood_code']

In [45]:
#set it to categorical
y = y.astype('category')

In [63]:
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [66]:
len(y_test_standard)

5402

In [67]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [68]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred))


              precision    recall  f1-score   support

           0       0.21      0.30      0.24       525
           1       0.24      0.55      0.33       199
           2       0.09      0.66      0.16       136
           3       0.07      0.46      0.12       134
           4       0.11      0.38      0.17       124
           5       0.14      0.24      0.18       275
           6       0.38      0.61      0.47       724
           7       0.77      0.07      0.13      3285

    accuracy                           0.22      5402
   macro avg       0.25      0.41      0.23      5402
weighted avg       0.56      0.22      0.20      5402



In [70]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.559     0.370     0.445       525
           1      0.343     0.839     0.487       199
           2      0.124     0.838     0.215       136
           3      0.113     0.739     0.196       134
           4      0.195     0.621     0.297       124
           5      0.307     0.473     0.372       275
           6      0.924     0.771     0.840       724
           7      0.979     0.402     0.570      3285

    accuracy                          0.492      5402
   macro avg      0.443     0.632     0.428      5402
weighted avg      0.812     0.492     0.556      5402



In [71]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [72]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM))

              precision    recall  f1-score   support

           0       0.20      0.38      0.27       525
           1       0.27      0.52      0.36       199
           2       0.10      0.62      0.18       136
           3       0.08      0.34      0.12       134
           4       0.07      0.57      0.13       124
           5       0.10      0.19      0.13       275
           6       0.44      0.34      0.38       724
           7       0.74      0.12      0.21      3285

    accuracy                           0.22      5402
   macro avg       0.25      0.38      0.22      5402
weighted avg       0.55      0.22      0.23      5402



In [73]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.536     0.450     0.489       525
           1      0.379     0.779     0.510       199
           2      0.130     0.750     0.221       136
           3      0.156     0.746     0.258       134
           4      0.092     0.661     0.161       124
           5      0.325     0.502     0.394       275
           6      0.965     0.677     0.795       724
           7      0.970     0.384     0.550      3285

    accuracy                          0.475      5402
   macro avg      0.444     0.619     0.422      5402
weighted avg      0.811     0.475     0.543      5402



In [74]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.20      0.25      0.22       525
           1       0.20      0.50      0.28       199
           2       0.09      0.49      0.15       136
           3       0.05      0.24      0.08       134
           4       0.05      0.25      0.09       124
           5       0.09      0.22      0.13       275
           6       0.34      0.38      0.36       724
           7       0.68      0.15      0.25      3285

    accuracy                           0.22      5402
   macro avg       0.21      0.31      0.19      5402
weighted avg       0.50      0.22      0.25      5402

              precision    recall  f1-score   support

           0      0.586     0.421     0.490       525
           1      0.287     0.739     0.413       199
           2      0.136     0.721     0.229       136
           3      0.106     0.537     0.177       134
           4      0.110     0.500     0.180       124
           5      0.215 

In [75]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.215     0.371     0.273       525
           1      0.226     0.648     0.335       199
           2      0.121     0.662     0.205       136
           3      0.071     0.433     0.122       134
           4      0.122     0.508     0.197       124
           5      0.164     0.211     0.185       275
           6      0.375     0.577     0.455       724
           7      0.704     0.082     0.147      3285

    accuracy                          0.237      5402
   macro avg      0.250     0.437     0.240      5402
weighted avg      0.524     0.237     0.211      5402

              precision    recall  f1-score   support

           0      0.579     0.482     0.526       525
           1      0.298     0.854     0.442       199
           2      0.156     0.831     0.263       136
           3      0.115     0.716     0.198       134
           4      0.176     0.661     0.277       124
           5      0.379 

In [76]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0       0.21      0.37      0.27       525
           1       0.23      0.63      0.34       199
           2       0.11      0.66      0.19       136
           3       0.07      0.44      0.12       134
           4       0.12      0.49      0.19       124
           5       0.17      0.22      0.20       275
           6       0.38      0.59      0.46       724
           7       0.74      0.07      0.14      3285

    accuracy                           0.23      5402
   macro avg       0.26      0.43      0.24      5402
weighted avg       0.55      0.23      0.21      5402

              precision    recall  f1-score   support

           0      0.553     0.459     0.502       525
           1      0.313     0.844     0.457       199
           2      0.147     0.838     0.251       136
           3      0.115     0.731     0.198       134
           4      0.180     0.661     0.283       124
           5      0.377 